In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wd
from ipywidgets import interact
import seaborn as sns
import ptitprince as pt
import re #I import this library 'cause I need to deal with regular expressions
from sklearn.decomposition import PCA
from mlxtend.plotting import plot_decision_regions
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn import tree
from yellowbrick.cluster import KElbowVisualizer
from sklearn import metrics
sns.set_style('whitegrid')
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from category_encoders import TargetEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import PowerTransformer # para usar Yeo-Johnson
from scipy.stats import skew
from sklearn.metrics import r2_score
from sklearn.feature_selection import mutual_info_regression
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

In [2]:
Api_file="Merged_Api.csv"
Api_df=pd.read_csv(Api_file)

In [5]:
Api_df['has_transitions']=Api_df['has_transitions'].astype('int')

In [6]:
Api_df

,id_band,score,name,has_transitions,albumCount,review_count
0,11143,0.500000,Cuntscrape,1,1.0,1
1,14768,0.770000,Psychiatric Regurgitation,1,1.0,1
2,16693,0.860000,Four Question Marks,1,2.0,1
3,16950,0.783333,Acranius,1,1.0,1
4,20572,0.550000,Emergency Gate,1,3.0,1
...,...,...,...,...,...,...
8090,60990,0.700000,Tidfall,0,1.0,4
8091,63005,0.700000,Emperor,0,1.0,4
8092,49377,0.680000,VARIOUS ARTISTS,0,1.0,6
8093,43600,0.680000,Valensia,0,1.0,6
